<a href="https://colab.research.google.com/github/Iispar/review-summary-API/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The model

This is the colab file for my machine learning model that predicts the sentiment value (stars) of an review dataset. Here you can see how the model has been made with explanations.

The dataset used is [amazon_reviews_multi](https://huggingface.co/datasets/amazon_reviews_multi). The model is a pretty basic one but 

In [2]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.or

# Preprocessing

The dataset includes reviews from multiple languages so we only import the english ones. The dataset also includes alot of useless data for us, we only need the reviews and their ratings so lets process everything else out.

In [3]:
dataset = datasets.load_dataset('amazon_reviews_multi', name='en'); # imports the dataset.
# check it works
print(dataset);

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})


In [4]:
dataset = dataset.shuffle() # shuffle the dataset for safety.
dataset = dataset.remove_columns(['review_id', 'product_id', 'reviewer_id', 'language', 'product_category']) # removes everything that we don't need
dataset = dataset.rename_column("stars", "label") # rename stars to label so it is a bit more understandable
# an error was coming up because of the labels were 1-5 and not 0-4 so let's change that for all.
# at the same time lets add the title to the start of the review with an :.

def addTitle_and_changeLables(example):
  example["label"] = example["label"] - 1; # lower the label by one so we get 0-4
  example["review_body"] = f'{example["review_title"]}: {example["review_body"]}'; # add title to review body
  return example # return the item
dataset = dataset.map(addTitle_and_changeLables) # map the function to all.
dataset = dataset.remove_columns(['review_title']) # now we can also remove the title

# let's check that it worked.
print(dataset)
print(dataset["train"][3]) 

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review_body'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['label', 'review_body'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'review_body'],
        num_rows: 5000
    })
})
{'label': 0, 'review_body': 'One Star: Product is just ok, but my real issue with it is that it arrived 2 days late.'}


# Vectorization

In [5]:
# vectorizes one item
def vectorize_item(item):
    vectorized = vectorizer.transform([item["review_body"]]); # vectorizes an item. Initialized below...
    non_zero_features = vectorized.nonzero()[1]; # get the nonzeros and we take only the columns of the nonzeros because our matrix is only one row.
    non_zero_features += 1; # index zero is for padding so let's avoid it by adding 1 to all.

    return {"input_ids":non_zero_features} 

In [7]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer( # get the vectorizer. try CountVectorizer also... 
    binary = True,
    max_features = 20000, # amount of words
    token_pattern = r"(?u)\b\w+\b", # take in one letter words also
    )

texts=[item["review_body"] for item in dataset["train"]]; # get all texts from train
vectorizer.fit(texts); # fitting the vectorizer

# vectorize the whole dataset.
dset_tokenized = dataset.map(vectorize_item,num_proc=4);

Map (num_proc=4):   0%|          | 0/200000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

# Padding and batching

In [8]:
def collator(list_of_items):
    allLabels = [item["label"] for item in list_of_items]; # list of all labels.
    batch = {"labels": torch.tensor(allLabels)}; # create a tenstor for the item (batch)
    tensors = [];
    max_len = max(len(item["input_ids"]) for item in list_of_items); # longest example in the batch. Pad to here.
    for item in list_of_items:
        ids = torch.tensor(item["input_ids"]); # input ids to tensor
        padded = torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])); # actual padding. Pads ids, from + to max with 0.
        tensors.append(padded); # appended ids to tensors
    batch["input_ids"] = torch.vstack(tensors); # stacks items as they are now same len. Now these are matrixes.
    return batch;

# check it works
batch=collator([dset_tokenized["train"][1],dset_tokenized["train"][16]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
print(batch["labels"])
print(batch["input_ids"])

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 26])
tensor([1, 3])
tensor([[19676, 19468, 19467, 19362, 19298, 17967, 16095, 16046, 15924, 14628,
         12077, 11936, 11769, 11476, 11472, 10345,  9383,  8113,  7195,  6935,
          6893,  3350,  2711,  1567,  1057,   513],
        [19899, 18857, 17967, 17906, 17705, 15640, 12765, 11968, 11964, 10422,
         10161,  9400,  9383,  8125,  7912,  7195,  6925,  5913,  3416,  2820,
          2523,  1057,   513,   493,     0,     0]])


# Machine learning model

## config

In [9]:
# needs a config, we wil just pass it.
class MLPConfig(transformers.PretrainedConfig):
    pass;

# model
class MLP(transformers.PreTrainedModel):
      config_class = MLPConfig; # sets config
      #initilazition
      def __init__(self,config):
        super().__init__(config); # call the super with out config which is now pass..
        self.vocab_size = config.vocab_size; # embedding matrix row count
        # Build embedding of vocab size +1 x hidden size. +1 again because of padding.
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0);
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001); # initialization of the embedding values
        self.output = torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels); # output layer is the size of the labels x hidden size.

      # forward
      def forward(self,input_ids,labels=None):
        embedded = self.embedding(input_ids); # sum up all the embeddings
        embedded_summed = torch.sum(embedded,dim=1); # sum up across word dimension
        projected = torch.tanh(embedded_summed); # non-linearity
        logits = self.output(projected); # apply the outer layer
      
        if labels is not None:
            # calculates the loss.
            loss = torch.nn.CrossEntropyLoss();
            return (loss(logits,labels),logits);
        else:
            # if no labels, just return the logits
            return (logits,);
  
# config
mlp_config = MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=25,nlabels=5); # hidden size 25 seems to be best, and labels 5 (1-5 stars)
     

## training

In [10]:
# training

# Set training arguments. Optimization happens soon...
trainer_args = transformers.TrainingArguments(
    output_dir="reviews_basic",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    eval_steps = 500,
    logging_steps = 500,
    learning_rate = 1e-4,
    max_steps = 20000,
    load_best_model_at_end = True,
    per_device_train_batch_size = 128
)


# evaluating
accuracy = evaluate.load("accuracy");
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels); # calc accuracy

# actual training
mlp = MLP(mlp_config); # Make a the actual model  
early_stopping = transformers.EarlyStoppingCallback(5); # stop training if the eval loss is not getting better.

# params
trainer = transformers.Trainer(
    model = mlp,
    args = trainer_args,
    train_dataset = dset_tokenized["train"],
    eval_dataset = dset_tokenized["test"],
    compute_metrics = compute_accuracy,
    data_collator = collator,
    callbacks = [early_stopping]
)

# train!
trainer.train();


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.538600,1.445400,0.527600
1000,1.342400,1.255573,0.548200
1500,1.188400,1.147188,0.559600
2000,1.101500,1.091117,0.566800
2500,1.052200,1.057512,0.575400
3000,1.024900,1.036502,0.578600
3500,0.995500,1.022191,0.581800
4000,0.983300,1.012577,0.583400
4500,0.967500,1.004671,0.583200
5000,0.957900,0.998873,0.587400


# Hyperparam optimization

In [11]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 64, 128])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 6),

    # params
    trainer_args = transformers.TrainingArguments(
        output_dir="reviews_basic",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 30000, # should reach the best result with 30000 steps and eval 500...
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs,
    )

    # the model again
    mlp = MLP(mlp_config)

    # train the model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = dset_tokenized["train"],
        eval_dataset = dset_tokenized["validation"],
        compute_metrics = compute_accuracy,
        data_collator = collator,
        callbacks = [early_stopping]
    )

    trainer.train() # train it again
    eval_results = trainer.evaluate() # get results
    return eval_results["eval_accuracy"] # return the best result.

# call the optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


[I 2023-04-23 18:14:16,501] A new study created in memory with name: no-name-61cdf270-4acf-4bef-b7a1-3a5442a4137c
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.610600,1.611103,0.210000
1000,1.610600,1.610693,0.221200
1500,1.609000,1.610291,0.228200
2000,1.611200,1.609897,0.237000
2500,1.609300,1.609518,0.244800
3000,1.609200,1.609144,0.250000
3500,1.608600,1.608773,0.254400
4000,1.607900,1.608405,0.257200
4500,1.608100,1.608043,0.260600
5000,1.607700,1.607679,0.264400


[I 2023-04-23 18:22:02,189] Trial 0 finished with value: 0.274 and parameters: {'learning_rate': 9.233228205554723e-07, 'batch_size': 8, 'num_train_epochs': 5}. Best is trial 0 with value: 0.274.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.610200,1.607820,0.202800
1000,1.604800,1.602155,0.206200
1500,1.600100,1.596282,0.221600
2000,1.592600,1.590137,0.237400
2500,1.586500,1.583491,0.266000
3000,1.579900,1.576854,0.283400
3500,1.573700,1.569896,0.326600
4000,1.566300,1.562704,0.356800
4500,1.559300,1.555428,0.373400
5000,1.552400,1.548011,0.394800


[I 2023-04-23 18:29:12,081] Trial 1 finished with value: 0.542 and parameters: {'learning_rate': 1.031643278754998e-05, 'batch_size': 16, 'num_train_epochs': 2}. Best is trial 1 with value: 0.542.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[I 2023-04-23 19:19:37,614] Trial 7 finished with value: 0.2 and parameters: {'learning_rate': 1.0675121687312777e-07, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 3 with value: 0.5842.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.611100,1.611452,0.200000
1000,1.611200,1.610806,0.200000
1500,1.610000,1.610182,0.200000
2000,1.609500,1.609561,0.200000
2500,1.609100,1.608951,0.200000
3000,1.608600,1.608348,0.200000
3500,1.608500,1.607773,0.200000
4000,1.607100,1.607193,0.200200
4500,1.606600,1.606632,0.200000
5000,1.606100,1.606073,0.200000


[I 2023-04-23 19:27:22,906] Trial 8 finished with value: 0.2088 and parameters: {'learning_rate': 7.125409238723389e-07, 'batch_size': 64, 'num_train_epochs': 6}. Best is trial 3 with value: 0.5842.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.434700,1.237147,0.534800
1000,1.135500,1.082297,0.553200
1500,1.048100,1.035936,0.567800
2000,1.016000,1.014957,0.572600
2500,0.989800,1.002062,0.576200
3000,0.972600,0.995420,0.578600
3500,0.939600,0.992008,0.579600
4000,0.932800,0.990185,0.582800
4500,0.932300,0.985034,0.584200
5000,0.916000,0.985407,0.580400


[I 2023-04-23 19:29:47,783] Trial 9 finished with value: 0.5804 and parameters: {'learning_rate': 0.0002457097934553275, 'batch_size': 64, 'num_train_epochs': 2}. Best is trial 3 with value: 0.5842.


# Evaluation

In [12]:
eval_results = trainer.evaluate(dset_tokenized["validation"]) # calc the accuracy
print(eval_results)

{'eval_loss': 0.9874362349510193, 'eval_accuracy': 0.5826, 'eval_runtime': 1.6063, 'eval_samples_per_second': 3112.744, 'eval_steps_per_second': 389.093, 'epoch': 10.88}


# Saving the model

Let's try it out with an dataset of items...
Even though the result is only 59% I think it will work in a larger dataset... let's see..

In [16]:
review_dset = datasets.load_dataset('amazon_us_reviews', name='Home_Entertainment_v1_00'); # imports the dataset.
# check it works
print(review_dset);

Generating train split:   0%|          | 0/705889 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/.cache/huggingface/datasets/amazon_us_reviews/Home_Entertainment_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date'],
        num_rows: 705889
    })
})


In [17]:
id = review_dset["train"][29]['product_id'] # get the id of the item we are reviewing
ratings = 0;
reviews = []
# let's just loop it thru and match to see how well it works and optimize later in actual backend...
for review in review_dset["train"]:
  if id == review['product_id']:
    ratings += review['star_rating'];
    reviews.append(review['review_body'])
  if len(reviews) >= 50: # match first 50.
    break;

print(len(reviews));

50


In [18]:
print(ratings/len(reviews)) # actual rating value

4.42


In [21]:
# creates a dataset from the review texts
def create_example(texts):
  labels = [];
  for item in texts:
    labels.append(0); # mark all labels as unknown so we can predict them
  data = {
      'review_body': texts,
      'label': labels,
  }
  return datasets.Dataset.from_dict(data);

In [22]:
# calculates the ratings from the predictions again just loop lets do this better in the backend...
auto_reviews = 0;
labels = [1, 2, 3, 4, 5];
example = create_example(reviews);
example_tokenized = example.map(vectorize_item, num_proc=4); # tokenize
for i in range(len(reviews)):
  prediction = trainer.predict(example_tokenized).predictions[i]; # predicts the label
  largest = max(prediction); # label with largest value
  labelOfLargest = labels[list(prediction).index(largest)]; # name of label
  auto_reviews += labelOfLargest;

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

In [23]:
print(auto_reviews/len(reviews));

4.22


Seems to be working and works pretty well atleast for this test.